<h1> Machine Learning - Test 2</h1>

<h2> Auteurs </h2>

* Forestier Quentin
* Herzig Melvyn


<h3> Importations </h3>
La première partie consiste à importer PyEvolve de manière à pouvoir utiliser notre algorithme génétique puis d'importer GeoPy pour calculer la distance entre les villes en tenant compte de la forme de la terre.

In [6]:
# For GA algorithm
from pyevolve import G1DBinaryString
from pyevolve import GSimpleGA
from pyevolve import Selectors
from pyevolve import Statistics

# For cites distances
from geopy.distance import geodesic as GD

<h3> Matrice des distances </h3>
Dans cette première, nous créons la matrice de distance à partir des latitudes et longitudes des villes. La matrice de distance est importante car elle sera utilisée plus tard dans la fonction de "fitness".

Pour ce faire, nous zippons les listes de coordonnées pour obtenir un tuple (lat, long) par ville. Ensuite, nous créons la matrices de distance à partir de cette liste de tuples. 


In [18]:
# Cities latitudes
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 
       17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

# Cities longitudes
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 
       96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

def compute_distances_matrix(coordinates):
    '''
    Compute a distance matrix based on list of (lat, lon).
    The cell [i, j] is the distance from the coordinate i to the coordinate j
    This is not an euclidean disance, but a distance that consider the earth spherical form.
    '''
    matrix={}
    for i, coordinate_src in enumerate(coordinates):
        for j, coordinate_dist in enumerate(coordinates):
            matrix[i, j] = GD(coordinate_src, coordinate_dist).km
    return matrix

distances_matrix = compute_distances_matrix(list(zip(LAT, LON)))

{(0, 0): 0.0, (0, 1): 177.25529290513245, (0, 2): 549.6979724345897, (0, 3): 715.2313688294823, (0, 4): 977.088097680002, (0, 5): 612.1690654621124, (0, 6): 453.276938991759, (0, 7): 83.28646194097563, (0, 8): 138.0263729638119, (0, 9): 344.64296013835855, (0, 10): 136.8190917723067, (0, 11): 561.5687481055201, (0, 12): 343.20248949914355, (0, 13): 432.8851371490596, (1, 0): 177.25529290513245, (1, 1): 0.0, (1, 2): 448.2003086963181, (1, 3): 664.884994171083, (1, 4): 1012.5921477784867, (1, 5): 635.0974923921588, (1, 6): 519.8449067576943, (1, 7): 213.07693556196085, (1, 8): 314.63150043394035, (1, 9): 477.42748393800184, (1, 10): 313.9540398462674, (1, 11): 571.951048460746, (1, 12): 432.52008728881805, (1, 13): 400.84443404820456, (2, 0): 549.6979724345897, (2, 1): 448.2003086963181, (2, 2): 0.0, (2, 3): 268.83362232049495, (2, 4): 746.3750160057954, (2, 5): 421.65770690527887, (2, 6): 469.85601380706834, (2, 7): 508.760318205063, (2, 8): 661.8059980794593, (2, 9): 893.9722829237367,

<h3> Fonction de fitness </h3>
Pour calculer la fitness d'un chromosome, nous calculons la distance totale du tour. 
Plus le tour est court meilleur est son "fitness".
Pour chaque index, nous calculons la distances au prochain index.
Ainsi pour l'index 0 nous obtenons la distance entre la ville à l'index 0 et la ville à l'index 1.
De la même manière, pour le dernier index, nous obtenons la distance jusqu'à la ville à l'index 0. 

In [8]:
def fitness_tour_length(chromosome_tour):
    global dist_matrix
    tour_length = 0.0
    nb_cities = len(chromosome_tour)
    for i in range(nb_cities):
        j = (i + 1) % nb_cities  
        score += dist_matrix[chromosome_tour[i], chromosome_tour[j]]
    return score
    

In [9]:
fitness(1)

[16.47, 16.47, 20.09, 22.39, 25.23, 22.0, 20.47, 17.2, 16.3, 14.05, 16.53, 21.52, 19.41, 20.09]
